%load_ext autoreload
%autoreload 2

In [38]:
%load_ext autoreload
%autoreload 2

import sys
import os
import importlib

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import src.config
import src.transform
import src.data_loader
import src.metrics

# Примусово оновлюємо код з диску в пам'ять
importlib.reload(src.config)
importlib.reload(src.transform)
importlib.reload(src.data_loader)
importlib.reload(src.metrics)

from src.config import VSTConfig
from src.transform import VarianceStabilizer
from src.data_loader import SyntheticGenerator, ImageLoader
from src.metrics import NoiseEstimator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
class VSTExplorerApp:
    def __init__(self, default_noised='data/NOISED.tiff', default_original='data/ORIGINAL.tiff'):
        # --- State ---
        self.default_noised = default_noised
        self.default_original = default_original
        
        self.cached_gen_data = None
        self.last_gen_noise_val = -1
        
        # --- UI Initialization ---
        self._init_widgets()
        self._init_layout()
        
    def _init_widgets(self):
        style = {'description_width': 'initial'}
        
        self.w_source = widgets.Dropdown(
            options=[('Генератор (Simulation)', 'gen'), ('Файл (File)', 'file')],
            value='gen', description='Джерело:', style=style
        )
        
        self.w_path = widgets.Text(
            value=self.default_noised, placeholder='path/to/image.tiff',
            description='Шлях:', style=style, layout=widgets.Layout(width='300px')
        )
        
        self.w_a = widgets.FloatSlider(value=8.39, min=1.0, max=20.0, step=0.1, 
                                     description='Param a:', style=style, continuous_update=False)
        self.w_b = widgets.FloatSlider(value=1.2, min=1.05, max=5.0, step=0.05, 
                                     description='Param b:', style=style, continuous_update=False)
        
        self.w_noise_gen = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, 
                                             description='Gen Noise:', style=style, continuous_update=False)

        # Оновлена кнопка: тепер вона називається "Оновити / Скинути"
        self.btn_reset = widgets.Button(description='Force Refresh', icon='refresh', button_style='warning')
        self.btn_reset.on_click(self.reset_params)
        
        self.out_plot = widgets.Output()

    def _init_layout(self):
        row_ctrl = widgets.HBox([self.w_source, self.w_noise_gen, self.w_path])
        row_params = widgets.HBox([self.w_a, self.w_b, self.btn_reset])
        
        def on_mode_change(change):
            mode = change['new']
            if mode == 'gen':
                self.w_path.layout.display = 'none'
                self.w_noise_gen.layout.display = 'flex'
            else:
                self.w_path.layout.display = 'flex'
                self.w_noise_gen.layout.display = 'none'
        
        self.w_source.observe(on_mode_change, names='value')
        on_mode_change({'new': self.w_source.value})

        for w in [self.w_source, self.w_path, self.w_a, self.w_b, self.w_noise_gen]:
            w.observe(self.update, names='value')
            
        display(widgets.VBox([row_ctrl, row_params, self.out_plot]))
        self.update()

    def reset_params(self, b):
        # 1. Скидаємо параметри VST на дефолтні
        self.w_a.value = 8.39
        self.w_b.value = 1.2
        
        # 2. !!! ВАЖЛИВО: Очищаємо кеш генератора !!!
        self.cached_gen_data = None
        self.last_gen_noise_val = -1
        
        # 3. Викликаємо оновлення вручну
        self.update()

    def get_data_pair(self):
        if self.w_source.value == 'gen':
            current_noise = self.w_noise_gen.value
            # Тепер ми генеруємо дані, якщо кеш пустий (після Reset) АБО змінився слайдер
            if self.cached_gen_data is None or current_noise != self.last_gen_noise_val:
                # Тут Python викличе оновлений код з файлу (завдяки %autoreload)
                self.cached_gen_data = SyntheticGenerator.get_data(noise_level=current_noise)
                self.last_gen_noise_val = current_noise
            return self.cached_gen_data
            
        else: # File mode
            try:
                path_in = self.w_path.value
                img_noised = ImageLoader.load_file(path_in)
                img_clean = None
                
                if os.path.exists(self.default_original):
                     img_clean = ImageLoader.load_file(self.default_original)
                
                if img_clean is None:
                    dir_name = os.path.dirname(path_in)
                    candidate_path = os.path.join(dir_name, 'ORIGINAL.tiff')
                    if os.path.exists(candidate_path):
                        img_clean = ImageLoader.load_file(candidate_path)

                return img_clean, img_noised
            except Exception:
                return None, None

    def update(self, change=None):
        img_clean, img_noised = self.get_data_pair()
        
        with self.out_plot:
            clear_output(wait=True)
            if img_noised is None:
                print(f"File error: {self.w_path.value}")
                return

            config = VSTConfig(a=self.w_a.value, b=self.w_b.value)
            vst = VarianceStabilizer(config)
            
            img_log_noised = vst.forward(img_noised)
            img_restored = vst.inverse(img_log_noised)
            
            sigma_blind = NoiseEstimator.estimate_blind_sigma(img_log_noised)
            sigma_exact = 0.0
            noise_map_vector = None
            
            if img_clean is not None:
                if img_clean.shape == img_noised.shape:
                    img_log_clean = vst.forward(img_clean)
                    sigma_exact = NoiseEstimator.calculate_exact_sigma(img_log_noised, img_log_clean)
                    noise_map_vector = (img_log_noised - img_log_clean).flatten()

            fig = plt.figure(figsize=(14, 8), constrained_layout=True)
            gs = fig.add_gridspec(2, 2)
            
            ax_in = fig.add_subplot(gs[0, 0])
            ax_log = fig.add_subplot(gs[0, 1])
            ax_out = fig.add_subplot(gs[1, 0])
            ax_hist = fig.add_subplot(gs[1, 1])
            
            vmin, vmax = np.percentile(img_noised, 1), np.percentile(img_noised, 99)
            
            im0 = ax_in.imshow(img_noised, cmap='gray', vmin=vmin, vmax=vmax)
            ax_in.set_title(f"Input\nMin: {img_noised.min():.2f}, Max: {img_noised.max():.2f}")
            plt.colorbar(im0, ax=ax_in, fraction=0.046)
            
            im1 = ax_log.imshow(img_log_noised, cmap='viridis')
            ax_log.set_title(f"Log Domain\nBlind Sigma: {sigma_blind:.4f}")
            plt.colorbar(im1, ax=ax_log, fraction=0.046)
            
            im2 = ax_out.imshow(img_restored, cmap='gray', vmin=vmin, vmax=vmax)
            mse = np.mean((img_noised - img_restored)**2)
            ax_out.set_title(f"Restored\nMSE: {mse:.2e}")
            plt.colorbar(im2, ax=ax_out, fraction=0.046)
            
            if noise_map_vector is not None:
                ax_hist.hist(noise_map_vector, bins=100, density=True, alpha=0.6, color='dodgerblue', label='Actual')
                x_axis = np.linspace(noise_map_vector.min(), noise_map_vector.max(), 100)
                mean_val = np.mean(noise_map_vector)
                pdf = (1 / (sigma_exact * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_axis - mean_val) / sigma_exact)**2)
                ax_hist.plot(x_axis, pdf, 'r--', linewidth=2, label=f'Gauss $\sigma={sigma_exact:.4f}$')
                ax_hist.set_title("Noise Histogram")
                ax_hist.legend()
            else:
                ax_hist.text(0.5, 0.5, "No Reference", ha='center')
                ax_hist.axis('off')
            
            plt.show()

app = VSTExplorerApp()

<>:157: SyntaxWarning: invalid escape sequence '\s'
<>:157: SyntaxWarning: invalid escape sequence '\s'
C:\Users\pawla\AppData\Local\Temp\ipykernel_41208\3335751905.py:157: SyntaxWarning: invalid escape sequence '\s'
  ax_hist.plot(x_axis, pdf, 'r--', linewidth=2, label=f'Gauss $\sigma={sigma_exact:.4f}$')
